In [1]:
import pandas as pd
import glob
import json
import os

In [2]:
def make_small_tweets(tweet):
    '''
    Check if the tweet is retweet, then filter only the fields useful to the research
    '''
    
    if 'retweeted_status' in tweet.keys():
        
        original = {'created_at': tweet['retweeted_status']['created_at'],
                    'id_str': tweet['retweeted_status']['id_str'],
                    'full_text': tweet['retweeted_status']['full_text'],
                    'entities': tweet['retweeted_status']['entities'],
                    'source': tweet['retweeted_status']['source'],
                    'retweet': False,
                    'retweet_count': tweet['retweeted_status']['retweet_count'],
                    'favorite_count': tweet['favorite_count']}
        
        retweet = {'created_at': tweet['created_at'],
                   'id_str': tweet['id_str'],
                   'full_text': tweet['full_text'],
                   'truncated': tweet['truncated'],
                   'entities': tweet['entities'],
                   'source': tweet['source'],
                   'retweet': True,
                   'retweet_count': tweet['retweet_count'],
                   'favorite_count': tweet['favorite_count']}
        
        return (original, retweet)
    
    else:
        
        original = {'created_at': tweet['created_at'],
                    'id_str': tweet['id_str'],
                    'full_text': tweet['full_text'],
                    'entities': tweet['entities'],
                    'source': tweet['source'],
                    'retweet': False,
                    'retweet_count': tweet['retweet_count'],
                    'favorite_count': tweet['favorite_count']}

        return (original, None)

In [3]:
def check_keywords(tweet):
    keywords = ['vaccine','vaccines','#vaccine','#vaccines','corona vaccine','corona vaccines','#coronavaccine','#coronavaccines','pfizer',
                'biontech','moderna','pfizer-biontech','pfizer/biontech','pfizer biontech','#pfizerbiontech','covax',
                '#covax','sinopharm','sinovac','astrazeneca','sputnik v','gamaleya']
    
    for keyword in keywords:
        if keyword in tweet['full_text'].lower():
            return True
    
    if len(tweet['entities']['hashtags']) > 0:
        for keyword in keywords:
            if keyword in [x.lower() for x in tweet['entities']['hashtags'][0]['text']]:
                return True
    else:
        return False

In [40]:
for folder_name in glob.glob('./CollectedTweets/308'): #Change to process multiple folders
    day_number = folder_name[-3:]
    parent_dir = "../ProcessedTweets/"
    path = os.path.join(parent_dir, day_number) 
    os.mkdir(path) 
    
    
    for file_name in glob.glob(folder_name + '/*.csv'):
        all_processed_tweets = []
        
        tweets = pd.read_csv(file_name)
        for tweet in tweets['json']:
            try:
                tweet_object = json.loads(tweet)
            except:
                print('Error')
                
            small_tweets = make_small_tweets(tweet_object) #Filtering the selected fields
            
            if check_keywords(small_tweets[0]):  #Checking if tweet has the keywords
                for small_tweet in small_tweets:
                    if small_tweet:
                        all_processed_tweets.append(small_tweet)
                
                
        with open('../Processedtweets/' + day_number + '/' + file_name.split('/')[-1].split('.')[0] + '.json', 'w') as outfile:
            json.dump(all_processed_tweets, outfile)
    
    print(f'Day {day_number} processed.')

Day 308 processed.
